<h1>Group 1 - Facebook<span class="tocSkip"></span>

# Introduction

In [ ]:
V0 : basic scrap

V1 : Feature adding

V2...

# Environnement

## Libraries


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common import action_chains
import re
import time
from datetime import datetime, timedelta,da

## Functions


This part contains all the functions we developped during the project

In [161]:
def dynamicSoup(url, link_driver, n_loop):
    """Documentation    
       Parameters:
            url : url page
            link_driver : driver link
            n_boucle : scroll time number
       out : 
            bs(driver.page_source, 'html.parser') : driver of the new page
            driver : driver
    """
    driver = webdriver.PhantomJS(link_driver)
    driver.set_window_size(1400,1000)
    driver.get(url)
    y = 0
        
    for _ in range(n_loop):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo("+str(y)+","+str(y+2000)+")")
        time.sleep(2)
        y += 2000
    return bs(driver.page_source, 'html.parser'), driver

In [162]:
def clickShow(driver):
    """Documentation    
       Parameters:
            driver : driver

    """
    driver.find_element_by_class_name("_3j0u").click()
    elements = driver.find_elements_by_class_name("_4ssp")
    action = action_chains.ActionChains(driver)
    for el in elements:
        time.sleep(2)
        try:
            action.move_to_element(el)
            time.sleep(4)
            action.click(el)
            action.perform()
        except:
            print("err")

In [164]:
def replace_all(text, dic):
    """Documentation    
       Parameters:
            text : character string
            dic : dictionary which contains the changes to be made
       out : 
            text : text with all changes made 
    """
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
def Simplification(chain):
    # transform M into 1000000,  and K into 1000
    """Documentation    
       Parameters:
            chain : character string in format "ex : 300K"    
       out : 
            int(float(expression) * coef) : integer  "ex 300 000"          
    """
    chain = chain.replace(',', '.')
    if '.' in chain:
        expression = (re.search("\d+\.\d+", chain)).group(0)
    else:
        expression = (re.search("\d+", chain)).group(0)
    coef = 1
    if 'K' in chain:
        coef = 1000
    if 'M' in chain:
        coef = 1000000
    return(int(float(expression) * coef))


## Create JSON file

In [242]:
def CreateJs(url_page):
    """Documentation    
       Parameters:
            url1_page
       out : 
            bs(driver.page_source, 'html.parser') : driver of the new page
            driver : driver
    """
    name_col = ['Data_Source', 'Airline_Name', 'Airline_Type', 'Region_Operation', 'Aircraft_Type', 'Cabin_Class', 'Type_Of_Lounge',
                'Type_Of_Traveller', 'Date_Visit', 'Date_Flown', 'Airport', 'Route', 'Category', 'Category_Detail',
                'Cabin_Staff_Service', 'Lounge_Staff_Service', 'Bar_And_Beverages', 'Food_And_Beverages', 'Ground_Service', 'Catering', 'Cleanliness',
                'Lounge_Comfort', 'Aisle_Space', 'Wifi_And_Connectivity', 'Inflight_Entertainment', 'Viewing_Tv_Screen', 'Power_Supply',
                'Seat', 'Seat_type', 'Seat_Comfort', 'Seat_Legroom', 'Seat_Storage', 'Seat_Width', 'Seat_Recline', 'Washrooms',
                'Value_For_Money', 'Overall_Customer_Rating', 'Overall_Service_Rating', 'Overall_Airline_Rating',
                'Recommended', 'Departure_city', 'Arrival_city', 'Nb_bus_taken', 'Nb_train_taken',
                'Nb_car_taken', 'Nb_plane_taken', 'Duration', 'Price_min', 'Price_max', 'Nb_sharing', 'Awards', 'Registration', 'Language',
                'Queuing Times', 'Terminal_Seating', 'Terminal Signs', 'Airport_Shopping', 'Experience_At_Airport', 'Date_Review']

    dataAirline = pd.DataFrame(columns=name_col)
    lien_driver = "C:/Users/neima/Downloads/Projet/ProjetInterpromo2020/Code/phantomjs-2.1.1-windows/bin/phantomjs.exe"
    soup, driver = dynamicSoup(url_page, lien_driver, 4)
    
    dic = {'.':'','avril':'04','janvier':'01',
           'mars':'03','mai':'05','juin':'06','février':'02','juillet':'07','décembre':'12',
           
           'août':'08','septembre':'09','aoÃ»t':'08','dÃ©c':'12', 'novembre':'11'}

    
    #get likes 
    nb_like = soup.find_all(
                 'span', attrs={'class':'_81hb'})
    nb_like2=[]
    for i in range (0,len(nb_like),2):
                 nb_like2.append(Simplification(nb_like[i].text.strip().replace('\xa0','')))

   


    #get description 
    for i in range (0,len(description)):
                 description[i]=description[i].strip()

    
    
    
    #get Date publication
    date1 = soup.find_all('span', attrs = {'class' : 'timestampContent' , 'id' : re.compile(r'js_')})
    for i in range (0,len(date1)):
                if('h' in (date1[i].text.strip()) ):
                    date1[i]=str(date.today())
                else :
                    date1[i]=replace_all(date1[i].text.strip(),dic)
                    if (re.search("^[0-9]*\s[0-9]*,", date1[i]))!=None :
                          date1[i]=date1[i].split(",")[0]+' 2020'
                    date1[i]=str(datetime.strptime(re.search("[0-9]* [0-9]* [0-9]*", date1[i]).group(0), '%d %m %Y')).split(" ")[0]
    
    
    #get share
    share = soup.find_all('a', attrs = {'class' : '_3rwx _42ft'})
    for i in range (0,len(share)):
        share[i]=Simplification(share[i].text.strip().replace("partages","").replace("\xa0","").replace(" ",""))
        
        
    #get nb_com
    nb_com = soup.find_all('a', attrs = {'class' : '_3hg- _42ft'})
    for i in range (0,len(nb_com)):
        nb_com[i]=Simplification(nb_com[i].text.strip().replace(" commentaires",""))
        
    
    #get comment
    comments= driver.find_elements_by_xpath("//div [@data-testid='UFI2Comment/body']")
    for i in range (len(comments)):
        comments[i]=comments[i].text
        
         
        
    #à corriger : 
        #il manque qlq publications qui ne sont méme pas dans le soup
        
    df_template = pd.DataFrame({}) #a faire
            dataAirline = pd.concat([dataAirline, df_template])  
    
    return(nb_like2, description,date1,share,nb_com,comments)

In [245]:
#Read the facebook pages to be scrapped in the pagesFb file, then scrape them one by one.
with open('C:/Users/neima/Downloads/Projet/ProjetInterpromo2020/Code/pagesFb', "r") as f :
    
     
    fb_pages = f.read()
    files = fb_pages.split("\n")
    
i=0
for page_link in files :
    
    #call function to scrape a page ex : scrape(link)
    CreateJs(page_link) 
    i+=1
    print('page numéro : ', i, ' fini')
print('extraction complete')



page numéro :  1  fini
page numéro :  2  fini
page numéro :  3  fini
page numéro :  4  fini
page numéro :  5  fini
page numéro :  6  fini
page numéro :  7  fini
page numéro :  8  fini
page numéro :  9  fini
page numéro :  10  fini
page numéro :  11  fini
page numéro :  12  fini
page numéro :  13  fini
page numéro :  14  fini
page numéro :  15  fini
page numéro :  16  fini
page numéro :  17  fini
page numéro :  18  fini
page numéro :  19  fini
page numéro :  20  fini
page numéro :  21  fini
page numéro :  22  fini
page numéro :  23  fini
page numéro :  24  fini
page numéro :  25  fini
page numéro :  26  fini


ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))